本文件将互联网上爬取来的文章（包含图片），通过NLP工具（LAC）对其进行NER标注处理，最终生成NE字典文件，标注好的训练数据

In [8]:
import numpy as np
import pandas as pd
import json
from pathlib import Path

from itertools import groupby


ImportError: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy c-extensions failed.
- Try uninstalling and reinstalling numpy.
- If you have already done that, then:
  1. Check that you expected to use Python3.7 from "D:\Users\Administrator\Anaconda3\envs\pytorch15\python.exe",
     and that you have no directories in your PATH or PYTHONPATH that can
     interfere with the Python and numpy version "1.17.4" you're trying to use.
  2. If (1) looks fine, you can open a new issue at
     https://github.com/numpy/numpy/issues.  Please include details on:
     - how you installed Python
     - how you installed numpy
     - your operating system
     - whether or not you have multiple versions of Python installed
     - if you built from source, your compiler versions and ideally a build log

- If you're working with a numpy git repository, try `git clean -xdf`
  (removes all files not under version control) and rebuild numpy.

Note: this error has ma

In [7]:
from LAC import LAC
lac = LAC(mode='lac')

ImportError: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy c-extensions failed.
- Try uninstalling and reinstalling numpy.
- If you have already done that, then:
  1. Check that you expected to use Python3.7 from "D:\Users\Administrator\Anaconda3\envs\pytorch15\python.exe",
     and that you have no directories in your PATH or PYTHONPATH that can
     interfere with the Python and numpy version "1.17.4" you're trying to use.
  2. If (1) looks fine, you can open a new issue at
     https://github.com/numpy/numpy/issues.  Please include details on:
     - how you installed Python
     - how you installed numpy
     - your operating system
     - whether or not you have multiple versions of Python installed
     - if you built from source, your compiler versions and ideally a build log

- If you're working with a numpy git repository, try `git clean -xdf`
  (removes all files not under version control) and rebuild numpy.

Note: this error has ma

In [2]:
import re
def cut_sent(para):
    para = re.sub('([。！？\?])([^”’])', r"\1\n\2", para)  # 单字符断句符
    para = re.sub('(\.{6})([^”’])', r"\1\n\2", para)  # 英文省略号
    para = re.sub('(\…{2})([^”’])', r"\1\n\2", para)  # 中文省略号
    para = re.sub('([。！？\?][”’])([^，。！？\?])', r'\1\n\2', para)
    # 如果双引号前有终止符，那么双引号才是句子的终点，把分句符\n放到双引号后，注意前面的几句都小心保留了双引号
    para = para.rstrip()  # 段尾如果有多余的\n就去掉它
    # 很多规则中会考虑分号;，但是这里我把它忽略不计，破折号、英文双引号等同样忽略，需要的再做些简单调整即可。
    return para.split("\n")

### 加载数据，包括Excel文件及其对应图片文件

In [3]:
renmin_data_path_dir = "./renmin_data"
wangyi_data_path_dir = "./wangyi_data"
renmin_fileName2df = {filePath.name.strip(filePath.suffix):pd.read_excel(filePath) for filePath in Path(f"{renmin_data_path_dir}/article_data").iterdir() if filePath.is_file()}
wangyi_fileName2df = {filePath.name.strip(filePath.suffix):pd.read_excel(filePath) for filePath in Path(f"{wangyi_data_path_dir}/article_data").iterdir() if filePath.is_file()}

NameError: name 'Path' is not defined

In [76]:
# 查看文章总数
count = 0
for name, df in renmin_fileName2df.items():
    print(name, ":", df.shape)
    count += df.shape[0]
print("人民日报文章总数：---------", count, "\n")

count = 0
for name, df in wangyi_fileName2df.items():
    print(name, ":", df.shape)
    count += df.shape[0]
print("网易文章总数：---------", count)

IT-2021-07-01 23-40-57 : (517, 3)
公益-2021-07-01 23-38-35 : (498, 3)
军事-中国军情-2021-07-01 23-43-06 : (481, 3)
军事-国防部-2021-07-01 23-41-51 : (201, 3)
军事-国际军情-2021-07-01 23-43-52 : (503, 3)
军事-本网原创-2021-07-01 23-42-20 : (424, 3)
军事-滚动新闻-2021-07-01 23-41-22 : (489, 3)
军事-热点解读-2021-07-01 23-44-14 : (77, 3)
军事-评论-2021-07-01 23-45-07 : (501, 3)
军事-高层动态-2021-07-01 23-41-37 : (38, 3)
娱乐-2021-07-01 23-40-02 : (487, 3)
房产-2021-07-01 23-35-50 : (80, 3)
房产-人民楼视-2021-07-01 23-36-40 : (96, 3)
房产-公司新闻-2021-07-02 00-07-18 : (94, 3)
教育-中小学-2021-07-01 23-47-48 : (515, 3)
教育-原创-2021-07-01 23-46-04 : (284, 3)
教育-大学-2021-07-01 23-48-22 : (504, 3)
教育-幼教-2021-07-01 23-47-17 : (509, 3)
教育-滚动新闻-2021-07-01 23-45-49 : (490, 3)
教育-留学-2021-07-01 23-46-46 : (518, 3)
教育-职业教育-2021-07-01 23-48-54 : (405, 3)
汽车-2021-07-02 00-07-21 : (0, 3)
游戏-2021-07-01 23-39-11 : (490, 3)
科技-2021-07-01 23-37-09 : (252, 3)
人民日报文章总数：--------- 8453 

NBA-2021-06-30 01-14-21 : (34, 3)
体育-2021-06-30 01-14-18 : (100, 3)
军事-2021-06-30 01-13-53 :

In [4]:
test_content = "6月11日\n，江苏天制环保科技有限公司(下称“天制环保科技公司”)向中华慈善总会捐赠天制垃圾无氧裂解热气化处理设备仪式在北京举行。中华慈善总会会长宫蒲光，荣誉副会长、新闻界慈善促进会执行会长兼秘书长徐镱轩，天制环保科技公司董事长孙文明、总经理霍玉琪出席捐赠仪式。仪式上，天制环保科技公司向中华慈善总会捐赠了价值5000万元的无氧裂解垃圾热气化处理设备，将用于乡村垃圾处理与环境治理。\
 宫蒲光对天制环保科技公司的慷慨善举表示衷心感谢与诚挚的敬意。他指出，践行社会责任是企业提升自身价值与产品口碑的良好途径。天制环保科技公司与中华慈善总会的合作就是巩固拓展脱贫攻坚成果与乡村振兴有效衔接的一次积极尝试，通过企业向广大农村地区捐赠垃圾处理环保设备，将与中华慈善总会“幸福家园”村社互助工程共同为助力乡村振兴事业添砖加瓦。双方将通过在全国范围内进行试点推广，发展一批适合各地特点的农业农村集体企业，促进县域经济，提升农村农业高质量发展。\
 徐镱轩在讲话中表示，垃圾治理是环境保护的重要环节，是助力乡村振兴的重要手段，也是中华慈善总会广泛开展“幸福家园”村社互助工程的重要利器。天制环保科技公司是一家积极履行社会责任的企业，其研发的垃圾处理设备既能使农村垃圾得到彻底治理，改善美化农村环境，又能变废为宝，使垃圾转变为电能热能服务社会，为乡村农产品深加工提供绿色廉价能源，助力乡村振兴战略。\
 孙文明在接受《慈善公益报》记者采访时表示，天制垃圾无氧裂解热气化处理设备安装之后，首先解决了农村环保问题。天制垃圾无氧裂解热气化处理系统是目前国内最适合县域农村环境治理的垃圾处理系统，可实现垃圾无害化和资源化再利用，转化为电能和热能，进一步为乡村集体企业提供大棚供暖，蔬菜和食用菌的种植、烘干等，成为促进农村农民增收，践行美丽中国环保理念的有益实践。"
# 装载LAC模型
def article_label(article_content = test_content):
    """
        采用LAC工具对文章内容article_content进行标注，同时返回
    """
    def convert_word_to_train_char(w, tag):
        res = []
        if w == "\n" or w == "。" or w == "！" or w == "？":
            res.append("\n")
            return res
        for i, char in enumerate(w):
            if tag.lower() == "o":
                res.append(f"{char} O")
            else:
                if i == 0:
                    res.append(f"{char} B-{tag.upper()}")
                else:
                    res.append(f"{char} I-{tag.upper()}")
        return res
    labeled_content_list = []
    type2entity = {}
    
    # 先将文档进行分句
    
    
    lac_result = lac.run(article_content)
    for word, label in zip(lac_result[0], lac_result[1]):
        if label in ["ORG","PER", "TIME", "LOC"]:
            type2entity.setdefault(label, [])
            type2entity[label].append(word)
            labeled_content_list.extend(convert_word_to_train_char(word, label))
        else:
            labeled_content_list.extend(convert_word_to_train_char(word, "O"))
    return type2entity, labeled_content_list
article_label()

NameError: name 'lac' is not defined

In [82]:
list(renmin_fileName2df.values())[0].head(1)

,title,content,imgs
0,陕西联通：落实乡村振兴战略 共建数字乡村生态,\n近日，中国联通召开数字乡村推进会，提出以数字化助力乡村全面振兴的举措。6月9日，陕西“数...,renmin_data\imgs\8ffca1b8-1e55-4640-b662-2c477...


In [ ]:
# 人民日报数据 将原始数据进行转化成可训练数据，同时统计相关数据
all_type2entity = {}  # 实体未去重
all_labeled_content_list = []
all_entity2imgs = {}
for name, df in renmin_fileName2df.items():
    if df.shape[0] != 0:
        for i,row in df.iterrows(): # 每篇文章
            row_dic = row.dropna().to_dict()
            imgs = row_dic.get('imgs', "")
            # 分割提取图片
            img_list = []
            for img_path in imgs.split(";"):
                if len(img_path) > 0: 
                    img_list.append(img_path)
                    
            # 文章内容
            content = row_dic.get("content", "")
            type2entity, labeled_content_list = article_label(str(content)) # labeled_content_list:["X O", "D B-ORG", ...]
            for type_, entities in type2entity.items():
                all_type2entity.setdefault(type_, [])
                all_type2entity[type_].extend(entities)
                
                # 为每个实体存储对应图片关系
                for entity in entities:
                    all_entity2imgs.setdefault(entity, [])
                    all_entity2imgs[entity].extend(img_list)
            
            # 保存每个文章转化后的训练数据
            all_labeled_content_list.append("\n".join(labeled_content_list))

d:\users\administrator\anaconda3\envs\pytorch15\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
print(len(all_type2entity), len(all_labeled_content_list), len(all_entity2imgs))

In [ ]:
# 网易数据 
for name, df in wangyi_fileName2df.items():
    if df.shape[0] != 0:
        for i,row in df.iterrows(): # 每篇文章
            row_dic = row.dropna().to_dict()
            imgs = row_dic.get('imgs', "")
            # 分割提取图片
            img_list = []
            for img_path in imgs.split(";"):
                if len(img_path) > 0: 
                    img_list.append(img_path)
                    
            # 文章内容
            content = row_dic.get("content", "")
            type2entity, labeled_content_list = article_label(str(content)) # labeled_content_list:["X O", "D B-ORG", ...]
            for type_, entities in type2entity.items():
                all_type2entity.setdefault(type_, [])
                all_type2entity[type_].extend(entities)
                
                # 为每个实体存储对应图片关系
                for entity in entities:
                    all_entity2imgs.setdefault(entity, [])
                    all_entity2imgs[entity].extend(img_list)
            
            # 保存每个文章转化后的训练数据
            all_labeled_content_list.append("\n".join(labeled_content_list))

In [ ]:
print(len(all_type2entity), len(all_labeled_content_list), len(all_entity2imgs))  # 未去重

In [ ]:
# all_entity2imgs 中的图片可能具有重复，对其进行统计， 排序（多到少）,去重
new_all_entity2imgs = {}
count_list = {}
for entity, imgs in all_entity2imgs.items():
    if len(imgs) != len(set(imgs)):
        temp = []
        imgs_sorted = sorted(imgs)
        for k, g in groupby(imgs_sorted):
            temp.append((k, len(list(g))))
        temp_sorted = [img for img, count in sorted(temp, key=lambda x: x[1], reverse = True)]
        new_all_entity2imgs[entity] = temp_sorted[20:]  # 出现次数较高的前几个图片大多为无意义通用图片（如图标，logo等）
    else:
        new_all_entity2imgs[entity] = imgs
    count_list[entity] = len(new_all_entity2imgs[entity])
# sorted(count_list.items(), key=lambda x: x[1], reverse=True)
all_entity2imgs = new_all_entity2imgs

In [ ]:
all_entity2imgs["黄河"]

In [ ]:
# 文件保存
def save_json_file(file_path, data): 
    with open(file_path, 'w', encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False)
save_json_file("./produced_data/all_type2entity.json", all_type2entity)
save_json_file("./produced_data/all_labeled_content_list.json", all_labeled_content_list)
save_json_file("./produced_data/all_entity2imgs.json", all_entity2imgs)

In [ ]:
# 集成训练数据，同统计句子长度
all_data = []
len_list = []
for article_labeled_content in all_labeled_content_list:
    sentence_tag_list = article_labeled_content.split("\n\n")
    for sentence_tag in sentence_tag_list:
        len_list.append(int((len(sentence_tag)/3)))
temp = sorted(len_list)
print(temp[1], " ", temp[-1])